## Numba

This approach aims to use numba to perform seamless integration between GPU and CPU code, with minimal C code conversions. It is obviously limited by how far numba can vectorize the code, but should be possible to provide high speeds with minimal programming effort.

In [19]:
from numba import vectorize, njit, cuda
import numpy as np
import math

In [43]:
# Returns the Lorentz transformed coordinate tuple. Uses cpu
@njit
def lorentzTransformation(x, y, z, t, v):
    c = 3*(10**8)
    rel_vel_op = math.sqrt(1-(v/c)**2)
    xt = (x - v*t)/rel_vel_op
    yt = y
    zt = z
    t = t-(v*x/c**2)/rel_vel_op
    return [xt, yt, zt, t]

# Automatic GPU vectorization (Specifically compilation)
@vectorize(["float32(float32, float32, float32)"], target='cuda')
def gauss_pdf(x, mean=0, stddev=1):
    return math.exp(((x-mean)/stddev)**2/2)/(math.sqrt(2*math.pi*stddev**2))

#CUDA Access. Relatively raw
@cuda.jit
def rayleigh_pdf(x, out, sigma=1):
    start = cuda.grid(1)          # 1D arrays
    strides = cuda.gridsize(1)
    for i in range(len(x)):
        out[i] = x[i]*math.exp(-(x[i]/sigma)**2/2)/sigma**2

In [51]:
x = np.random.randn(40000).astype(np.float32)
y = np.random.randn(40000).astype(np.float32)
z = np.random.randn(40000).astype(np.float32)
t = np.random.randn(40000).astype(np.float32)
v = 2.5*10**8
x_test = np.arange(100000).astype(np.float32)

In [52]:
out = cuda.device_array_like(x_test)
%lprun -f rayleigh_pdf rayleigh_pdf(x, out, 1)

/home/jaydeep/anaconda3/lib/python3.6/site-packages/line_profiler.py:321: UserWarning: Could not extract a code object for the object <numba.cuda.compiler.AutoJitCUDAKernel object at 0x7ff7c115ef98>
  profile = LineProfiler(*funcs)
